In [83]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [84]:
import os
import numpy as np

In [85]:
#text_file = open("/content/SW_EpisodeV.txt" , "r").read()

In [130]:

data = data2 = data3 = ""
with open("/content/SW_EpisodeIV.txt", "r") as f:
  data = f.read()
with open("/content/SW_EpisodeV.txt" , "r") as f2:
  data2 = f2.read()
with open("/content/SW_EpisodeVI.txt" , "r") as f3:
  data3 = f3.read()

data += "\n"
data = data + data2 + data3




In [133]:
text_file = data

In [134]:
print(f"length of text: {len(text_file)}")

length of text: 181942


In [135]:
text_file[:250]

'"character" "dialogue"\n"1" "THREEPIO" "Did you hear that?  They\'ve shut down the main reactor.  We\'ll be destroyed for sure.  This is madness!"\n"2" "THREEPIO" "We\'re doomed!"\n"3" "THREEPIO" "There\'ll be no escape for the Princess this time."\n"4" "THR'

In [137]:
vocab = sorted(set(text_file))
len(vocab)

76

In [138]:
#mapping char to int and int to char

char2idx = { u : i for i , u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text_file])

In [139]:
for char, _  in zip(char2idx , range(20)):
  print(f" {repr(char)} : {char2idx[char]}")


 '\n' : 0
 ' ' : 1
 '!' : 2
 '"' : 3
 '#' : 4
 "'" : 5
 ',' : 6
 '-' : 7
 '.' : 8
 '/' : 9
 '0' : 10
 '1' : 11
 '2' : 12
 '3' : 13
 '4' : 14
 '5' : 15
 '6' : 16
 '7' : 17
 '8' : 18
 '9' : 19


In [91]:
print(f"{repr(text_file[:10])} ---- characters mapped to int ---- >{text_as_int[:20]}")

'"character' ---- characters mapped to int ---- >[ 3 49 54 47 64 47 49 66 51 64  3  1  3 50 55 47 58 61 53 67]


In [140]:
seq_length = 100
examples_per_epoch = len(text_file)/(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

"
c
h
a
r
a
c
t
e
r


In [141]:
sequences = char_dataset.batch(seq_length+1 , drop_remainder= True)
for item in sequences.take(10):
  print(repr("".join(idx2char[item.numpy()])))

'"character" "dialogue"\n"1" "THREEPIO" "Did you hear that?  They\'ve shut down the main reactor.  We\'ll'
' be destroyed for sure.  This is madness!"\n"2" "THREEPIO" "We\'re doomed!"\n"3" "THREEPIO" "There\'ll be'
' no escape for the Princess this time."\n"4" "THREEPIO" "What\'s that?"\n"5" "THREEPIO" "I should have k'
'nown better than to trust the logic of a half-sized thermocapsulary dehousing assister..."\n"6" "LUKE"'
' "Hurry up!  Come with me!  What are you waiting for?!  Get in gear!"\n"7" "THREEPIO" "Artoo! Artoo-De'
'too, where are you?"\n"8" "THREEPIO" "At last!  Where have you been?"\n"9" "THREEPIO" "They\'re heading '
"in this direction. What are we going to do?  We'll be sent to the spice mines of Kessel or smashed in"
'to who knows what!"\n"10" "THREEPIO" "Wait a minute, where are you going?"\n"11" "IMPERIAL OFFICER" "Th'
'e Death Star plans are not in the main computer."\n"12" "VADER" "Where are those transmissions you int'
'ercepted?"\n"13" "REBEL OFFICER" "We intercep

In [142]:
def split_input_target(chunks):
  input_text = chunks[:-1]
  target = chunks[1:]
  return input_text, target

In [143]:
dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [144]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '"character" "dialogue"\n"1" "THREEPIO" "Did you hear that?  They\'ve shut down the main reactor.  We\'l'
Target data: 'character" "dialogue"\n"1" "THREEPIO" "Did you hear that?  They\'ve shut down the main reactor.  We\'ll'


In [145]:
#create training batches
batch_size = 64

shuffle_buffer = 10000

dataset = dataset.shuffle(shuffle_buffer).batch(batch_size , drop_remainder = True)

dataset



<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [146]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [147]:
def build_model(vocab_size , embedding_dim , rnn_units, batch_size ):
  model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size , embedding_dim , batch_input_shape = [batch_size , None]),
                             tf.keras.layers.GRU(rnn_units , return_sequences= True , stateful= True,
                                                 recurrent_initializer = "glorot_uniform" ),
                             tf.keras.layers.Dense(vocab_size)


  ])

  return model


In [148]:
model = build_model(vocab_size , embedding_dim , rnn_units , batch_size )
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (64, None, 256)           19456     
_________________________________________________________________
gru_10 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_9 (Dense)              (64, None, 76)            77900     
Total params: 4,035,660
Trainable params: 4,035,660
Non-trainable params: 0
_________________________________________________________________


In [149]:
for input_example_batch , target_example_batch in dataset.take(1):
  example_batch_prediction = model(input_example_batch )
  print(example_batch_prediction.shape , " #(batch , sequence_length , vocab_size)")



(64, 100, 76)  #(batch , sequence_length , vocab_size)


In [150]:
sample_indicies = tf.random.categorical(example_batch_prediction[0] , num_samples=1)
sample_indicies = tf.squeeze(sample_indicies , -1)
sample_indicies

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([31, 44, 21, 23, 61, 27, 30,  9, 65, 42,  2,  1, 28, 45, 52, 53, 26,
       30, 45, 17, 63, 17, 41, 54, 56,  8, 36, 17, 40, 48, 47, 51, 54, 33,
       19, 32,  2, 39, 44, 52, 24, 27, 21, 39,  9, 65, 42, 44, 65,  2, 73,
       49, 75, 59, 31, 15, 18, 36, 37, 11,  0, 26, 33, 33, 12,  6,  0, 10,
       17, 15, 26, 17, 66,  0, 17, 45, 70, 23, 18, 68, 59, 56, 38, 19, 24,
        1, 23, 10, 38, 19, 65, 15, 47, 30, 40,  9, 48, 52, 42, 58])>

In [151]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sample_indicies])))

Input: 
 ' reach Alderaan."\n"356" "HAN" "Seventeen, huh!"\n"357" "HAN" "Okay.  You guys got yourself a ship.  W'

Next Char Predictions: 
 'IV;AlEH/pT! FWcdDHW7n7Seg.N7RZYbeK9J!QVcBE;Q/pTVp!x\\zjI58NO1\nDKK2,\n075D7q\n7WuA8sjgP9B A0P9p5YHR/ZcTi'


In [152]:
def loss(labels , logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels ,logits , from_logits= True)

example_batch_loss  = loss(target_example_batch, example_batch_prediction)
print("Prediction shape: ", example_batch_prediction.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 76)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.332274


In [153]:
model.compile(loss = loss , optimizer = "adam" )

In [154]:
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir , "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_prefix , save_weights_only=True )

In [155]:
EPOCHS = 20


In [156]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
28/28 [==============================] - 1s 44ms/step - loss: 3.9723
Epoch 2/20
28/28 [==============================] - 1s 44ms/step - loss: 2.9212
Epoch 3/20
28/28 [==============================] - 1s 45ms/step - loss: 2.4010
Epoch 4/20
28/28 [==============================] - 1s 44ms/step - loss: 2.1598
Epoch 5/20
28/28 [==============================] - 1s 44ms/step - loss: 2.0511
Epoch 6/20
28/28 [==============================] - 1s 45ms/step - loss: 1.9705
Epoch 7/20
28/28 [==============================] - 1s 45ms/step - loss: 1.8947
Epoch 8/20
28/28 [==============================] - 1s 44ms/step - loss: 1.8220
Epoch 9/20
28/28 [==============================] - 1s 44ms/step - loss: 1.7552
Epoch 10/20
28/28 [==============================] - 1s 45ms/step - loss: 1.6909
Epoch 11/20
28/28 [==============================] - 1s 45ms/step - loss: 1.6327
Epoch 12/20
28/28 [==============================] - 1s 45ms/step - loss: 1.5771
Epoch 13/20
28/28 [==================

In [157]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [158]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [159]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (1, None, 256)            19456     
_________________________________________________________________
gru_11 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
dense_10 (Dense)             (1, None, 76)             77900     
Total params: 4,035,660
Trainable params: 4,035,660
Non-trainable params: 0
_________________________________________________________________


In [160]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [161]:
print(generate_text(model, start_string= "LUKE - I'm your dad."))

LUKE - I'm your dad. We anot mestake alson. Adait of pelminis ont one-   no doos belificated that plossible.  They're unive courselt.  Hey, whharg in the Force. Alrood you, founce reamy fromage, Buter."
"452" "HAN" "It is your carffort formather has. Leia!"
"393" "CREATURE" "We help you for take him instabielling for our supars."
"189" "LUKE" "You've been to five amperato consectrouth trouble to to to whis cannagien."
"117" "HAN" "Get!  You're a gie? Hey, But hid you!"
"595" "LEIA" "I'm going to me."
"545" "LANDO" "Stransportiance it were hem frol not. I didn't know! Artoo, un that  Gold you, Actol. Then I have to fell her..."
"132" "RED LEADER" "Princess dafle quitter, Luke."
"170" "LANDO" "But you can't get into power, wo did.  Sy don't be ready. I want tempure, to you are roygen. It is Oking him."
"147" "LAIU" "Bo they wos't be any our cruysina..."
"24" "Whore is not you can't chand you to  ald help!"
"138" "THREEPIO" "Our space can fail, of course."
"523" "THREEPIO" "I'm in her far